<a href="https://colab.research.google.com/github/akanksha-th/Books_Recommender/blob/main/Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
books = pd.read_csv("/content/books.csv", sep=';', on_bad_lines="skip", encoding='latin1')
print(f"Shape of 'books' tables : {books.shape}")
ratings = pd.read_csv("/content/ratings.csv", sep=';', on_bad_lines="skip", encoding='latin1')
print(f"Shape of 'ratings' tables : {ratings.shape}")
users = pd.read_csv("/content/users.csv", sep=';', on_bad_lines="skip", encoding='latin1')
print(f"Shape of 'users' tables : {users.shape}")

/tmp/ipykernel_11973/2316837463.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("/content/books.csv", sep=';', on_bad_lines="skip", encoding='latin1')


Shape of 'books' tables : (271360, 8)
Shape of 'ratings' tables : (1149780, 3)
Shape of 'users' tables : (278858, 3)


In [3]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [6]:
#users[users['Location'].str.split(",").str.len()!=3].value_counts()
drop_indices = users[users['Location'].str.split(",").str.len()!=3]
users.drop(drop_indices.index, inplace=True)

users[users['Location'].str.split(",").str.len()!=3].value_counts()

Series([], Name: count, dtype: int64)

In [7]:
users[['City', 'State', 'Country']] = users['Location'].str.strip().str.split(',', expand=True)
users.drop('Location', axis=1, inplace=True)
users.sample(12)

,User-ID,Age,City,State,Country
182865,182866,32.0,belo horizonte,minas gerais,brazil
30818,30819,17.0,maia,porto,portugal
130428,130429,NaN,sydney,new south wales,australia
44343,44344,22.0,allston,massachusetts,usa
269458,269459,43.0,salem,oregon,usa
249263,249264,50.0,london,uk,united kingdom
73223,73224,42.0,salt lake city,utah,usa
18902,18903,36.0,campobello,south carolina,usa
115102,115103,29.0,kowloon,n/a,hong kong
52510,52511,66.0,louisville,kentucky,usa


## DATA CLEANING AND PREPROCESSING

In [8]:
%%capture
!pip install skimpy
from skimpy import skim

In [9]:
print(f"SUMMARY OF 'BOOKS' DATAFRAME : shape={books.shape} \n"), skim(books)
print('\n', '-'*1000, '\n')
print(f"SUMMARY OF 'RATINGS' DATAFRAME : shape={ratings.shape} \n"), skim(ratings)
print('\n', '-'*1000, '\n')
print(f"SUMMARY OF 'USERS' DATAFRAME : shape={users.shape} \n"), skim(users)
print('\n', '-'*1000, '\n')

SUMMARY OF 'BOOKS' DATAFRAME : shape=(271360, 8) 



╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 271360 │ │ string      │ 7     │                                                          │
│ │ Number of columns │ 7      │ └─────────────┴───────┘                                                          │
│ └───────────────────┴────────┘                                                                                  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ ISBN                      │       0 │          0 │                            1 │                   271364 │  │
│ │ Book-Title                │       0 │          0 │                          6.1 │                  1664022 │  │
│ │ Book-Author               │       2 │          0 │                          2.2 │                   604798 │  │
│ │ Publisher                 │       2 │          0 │                          2.3 │                   631395 │  │
│ │ Image-URL-S               │       0 │          0 │                            1 │                   271360 │  │
│ │ Image-URL-M               │       0 │          0 │                            1 │                   271360 │  │
│ │ Image-URL-L               │       3 │          0 │                            1 │                   271357 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯


 --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                 Data Types                                                                │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                         │
│ ┃ dataframe         ┃ Values  ┃ ┃ Column Type ┃ Count ┃                                                         │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                         │
│ │ Number of rows    │ 1149780 │ │ int64       │ 2     │                                                         │
│ │ Number of columns │ 3       │ │ string      │ 1     │                                                         │
│ └───────────────────┴─────────┘ └─────────────┴───────┘                                                         │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name    ┃ NA  ┃ NA %   ┃ mean     ┃ sd      ┃ p0  ┃ p25     ┃ p50     ┃ p75     ┃ p100    ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩  │
│ │ User-ID        │   0 │      0 │   140400 │   80560 │   2 │   70340 │  141000 │  211000 │  278900 │ ▇▇▇▇▇▇  │  │
│ │ Book-Rating    │   0 │      0 │    2.867 │   3.854 │   0 │       0 │       0 │       7 │      10 │ ▇  ▁▂▂  │  │
│ └────────────────┴─────┴────────┴──────────┴─────────┴─────┴─────────┴─────────┴─────────┴─────────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ ISBN                      │       0 │          0 │                            1 │                  1149905 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯


 --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 277348 │ │ string      │ 3     │                                                          │
│ │ Number of columns │ 5      │ │ int64       │ 1     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name   ┃ NA       ┃ NA %   ┃ mean    ┃ sd     ┃ p0  ┃ p25    ┃ p50     ┃ p75     ┃ p100    ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩  │
│ │ User-ID       │        0 │      0 │  139400 │  80490 │   1 │  69720 │  139400 │  209100 │  278900 │ ▇▇▇▇▇▇ │  │
│ │ Age           │   110204 │  39.73 │   34.73 │  14.41 │   0 │     24 │      32 │      44 │     244 │   ▇▃   │  │
│ └───────────────┴──────────┴────────┴─────────┴────────┴─────┴────────┴─────────┴─────────┴─────────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name               ┃ NA      ┃ NA %       ┃ words per row                ┃ total words              ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ City                      │       0 │          0 │                          1.2 │                   339269 │  │
│ │ State                     │       0 │          0 │                          2.2 │                   607094 │  │
│ │ Country                   │       0 │          0 │                          2.1 │                   573512 │  │
│ └───────────────────────────┴─────────┴────────────┴──────────────────────────────┴──────────────────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯


 --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
books.duplicated().sum(), ratings.duplicated().sum(), users.duplicated().sum()

(0, 0, 0)

In [11]:
books.nunique(), books.shape

(ISBN                   271360
 Book-Title             242135
 Book-Author            102022
 Year-Of-Publication       202
 Publisher               16807
 Image-URL-S            271044
 Image-URL-M            271044
 Image-URL-L            271041
 dtype: int64,
 (271360, 8))

In [12]:
import re
pattern = r'[[";]]'
for index, row in books.iterrows():
    if re.search(r';', row['Book-Title']):
      if re.search(r'"', row['Book-Title']):
        print(f"Row: {index}, ISBN: {row['ISBN']}, Title: {row['Book-Title']}")

print('\n', '-'*100)
books[books.isnull().any(axis=1)]

Row: 14343, ISBN: 1930662513, Title: Intelligence, Internment &amp; Relocation: Roosevelt's Executive Order 9066: How Top Secret \MAGIC\" Intelligence Led to Evacuation"
Row: 20263, ISBN: 1570642389, Title: Barney &amp; Bj Go to the Police Station (Barney \Go to)"
Row: 90057, ISBN: 0809128136, Title: About Men &amp; Women: How Your \Great Story\" Shapes Your Destiny"
Row: 128623, ISBN: 1560975296, Title: Krazy &amp; Ignatz 1929-1930: \A Mice, A Brick, A Lovely Night\" (Krazy Kat)"
Row: 128625, ISBN: 1560975075, Title: Krazy &amp; Ignatz 1927-1928: \Love Letters in Ancient Brick\" (Krazy Kat)"
Row: 128627, ISBN: 1560973862, Title: Krazy &amp; Ignatz 1925-1926: \There is a Heppy Land Furfur A-waay\" (Krazy Kat)"
Row: 129182, ISBN: 0886779545, Title: Sword-Sworn: A Novel of Tiger and Del (Tiger &amp; Del \Sword\" # 6)"
Row: 142631, ISBN: 0688149987, Title: Proud Spirit: Lessons, Insights &amp; Healing from \the Voice of the Spirit World\""
Row: 163418, ISBN: 0965649709, Title: Property &a

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
118033,0751352497,A+ Quiz Masters:01 Earth,NaN,1999,Dorling Kindersley,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...
129037,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...
187689,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,NaN
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,NaN
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,NaN


### As you can notice, not are the values missing, but there is misalignment in the last 3 rows.

In [13]:
#After noticing patterns, the following 3 rows have the misalignment issue
books['Book-Title'][220731]
books['Book-Title'][221678]
books['Book-Title'][209538]

'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)\\";Michael Teitelbaum"'

In [14]:
col = books.columns
misaligned_rows = ['078946697X', '2070426769', '0789466953']

for index, row in books.iterrows():
    if row['ISBN'] in misaligned_rows:
        book_title_parts = row['Book-Title'].split(";")

        row['Image-URL-L'] = row['Image-URL-M']
        row['Image-URL-M'] = row['Image-URL-S']
        row['Image-URL-S'] = row['Publisher']
        row['Publisher'] = row['Year-Of-Publication']
        row['Year-Of-Publication'] = row['Book-Author']
        row['Book-Author'] = book_title_parts[-1]
        row['Book-Title'] = ";".join(book_title_parts[:-1])
        #print(index, row['ISBN'])
books[books.isnull().any(axis=1)]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
118033,0751352497,A+ Quiz Masters:01 Earth,NaN,1999,Dorling Kindersley,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002,NaN,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...
129037,1931696993,Finders Keepers,Linnea Sinclair,2001,NaN,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...
187689,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN,1995,Edinburgh Financial Publishing,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...


In [15]:
books.dropna(inplace=True)
books[books.isnull().any(axis=1)]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L


In [16]:
books[books['ISBN'].isin(['078946697X', '2070426769', '0789466953'])]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...","Michael Teitelbaum""",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\""","Jean-Marie Gustave Le ClÃ?Â©zio""",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...","James Buckley""",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...


In [17]:
books['Year-Of-Publication'] = pd.to_numeric(books['Year-Of-Publication'], errors='coerce')
books = books.dropna(subset=['Year-Of-Publication'])
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype(int)
books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271356 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271356 non-null  object
 1   Book-Title           271356 non-null  object
 2   Book-Author          271356 non-null  object
 3   Year-Of-Publication  271356 non-null  int64 
 4   Publisher            271356 non-null  object
 5   Image-URL-S          271356 non-null  object
 6   Image-URL-M          271356 non-null  object
 7   Image-URL-L          271356 non-null  object
dtypes: int64(1), object(7)
memory usage: 18.6+ MB


In [18]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]
books.sample(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
112586,0140188525,The Go-Between (Penguin Classics),L. P. Hartley,1997,Penguin USA,http://images.amazon.com/images/P/0140188525.0...
8608,0064405842,"Catherine, Called Birdy (Trophy Newbery)",Karen Cushman,1995,HarperTrophy,http://images.amazon.com/images/P/0064405842.0...


In [19]:
exclude = r'[?!@#$%^&*_;.\-\'\+=/\"[\]{}|\\]'
books['Book-Title'] = books['Book-Title'].astype(str).str.replace(exclude, '', regex=True)
print("Completed Successfully!!")

pd.DataFrame(books['Book-Title'].sample(10))

Completed Successfully!!


,Book-Title
187825,Don Quijote De LA Mancha I (Biblioteca Didacti...
237111,The Godless Constitution: The Case Against Rel...
119829,What Bird Is This
101667,From Fatigued to Fantastic
70119,Daughters (Plume Contemporary Fiction)
91825,Ohio: The Young Buckeye State Blossoms With Lo...
22989,To My Dear Slimeball
108111,Conexiones: Comunicacion y cultura
151608,Das neue Lexikon der populÃÂ¤ren IrrtÃÂ¼mer
259858,Mrs Frisby and the Rats of NimhNewbery Summer


In [20]:
exclude = r'[?!@#$%^&*_:;,.\-\'\+=/\"[\]{}|]'
books['Book-Author'] = books['Book-Author'].astype(str).str.replace(exclude, '', regex=True)

print("Completed Successfully!!")

pd.DataFrame(books['Book-Author'].sample(10))

Completed Successfully!!


,Book-Author
45748,Alyssa Dean
122362,William Taylor
204697,Rich Mullins
175157,Iona Archibald Opie
130903,Diki Tsering
175680,Lynda Barry
22792,Charles Dickens
217024,Rosi Braidotti
233367,Anthony Brooks
105688,Robert Barnard


In [21]:
ratings.nunique(), ratings.shape

(User-ID        105283
 ISBN           340556
 Book-Rating        11
 dtype: int64,
 (1149780, 3))

In [22]:
x = ratings['User-ID'].value_counts() > 200
x[x].shape

(899,)

In [23]:
y = x[x].index
ratings = ratings[ratings['User-ID'].isin(y)]
ratings.sample(3)

,User-ID,ISBN,Book-Rating
897033,217106,0387906525,0
330293,78783,0515119784,0
514724,124942,0060212268,0


In [24]:
ratings.nunique(), '-'*50+'\n', ratings.shape

(User-ID           899
 ISBN           207291
 Book-Rating        11
 dtype: int64,
 '--------------------------------------------------\n',
 (526356, 3))

In [25]:
users.nunique(), users.shape

(User-ID    277348
 Age           164
 City        32482
 State        5947
 Country       696
 dtype: int64,
 (277348, 5))

In [26]:
missing_in_users = users[users.isnull().any(axis=1)]
missing_percentage = (len(missing_in_users)/users.shape[0])*100
print(f"Total entries: {users.shape}\n",
      f"Total missing values: {len(missing_in_users)}\n",
      f"Percentage of missing values : {round(missing_percentage,2)}%")

Total entries: (277348, 5)
 Total missing values: 110204
 Percentage of missing values : 39.73%


In [27]:
missing_in_users.sample(10)

,User-ID,Age,City,State,Country
199300,199301,NaN,poteau,oklahoma,usa
184584,184585,NaN,plymouth,new hampshire,usa
33774,33775,NaN,toronto,ontario,canada
148338,148339,NaN,edmonton,alberta,canada
213961,213962,NaN,cary,north carolina,usa
21509,21510,NaN,bologna,emilia romagna,italy
82268,82269,NaN,reading,pennsylvania,usa
183760,183761,NaN,wellesley,massachusetts,usa
205854,205855,NaN,dublin,ireland,ireland
86428,86429,NaN,london,england,united kingdom


In [28]:
users.dropna(subset=['Age'], inplace=True)
users.sample()

,User-ID,Age,City,State,Country
260444,260445,27.0,&#304;stanbul,&#304;stanbul,turkey


### DATA MERGING

In [29]:
combined_df = books.merge(ratings, on='ISBN')
print(combined_df.shape)
combined_df.sample(2)

(487668, 8)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,User-ID,Book-Rating
217650,0393316009,Ship Fever,Andrea Barrett,1996,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393316009.0...,238120,0
193430,0446606022,Bad Chili,Joe R Lansdale,1998,Warner Books,http://images.amazon.com/images/P/0446606022.0...,148744,0


In [52]:
num_ratings = combined_df.groupby(by='Book-Title')['Book-Rating'].count().sort_values(ascending=False).reset_index()
#num_ratings = combined_df.groupby(by='Book-Title')['Book-Rating'].count().reset_index()
num_ratings.head()

,Book-Title,Book-Rating
0,Wild Animus,363
1,Bridget Joness Diary,277
2,The Lovely Bones: A Novel,270
3,The Notebook,241
4,The Pelican Brief,236


In [53]:
num_ratings.describe()

,Book-Rating
count,159958.000000
mean,3.048725
std,7.437167
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,363.000000


In [54]:
num_ratings.rename(columns={'Book-Rating' : 'Number_of_Ratings'}, inplace=True)

In [55]:
final_rating = combined_df.merge(num_ratings, on='Book-Title')
print(final_rating.shape)
final_rating.sample(3)

(487668, 9)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,User-ID,Book-Rating,Number_of_Ratings
359008,0843951192,Walk into the Flame,Ronda Thompson,2003,Leisure Books,http://images.amazon.com/images/P/0843951192.0...,227447,0,4
198288,0440416639,The Cay,THEODORE TAYLOR,2002,Yearling,http://images.amazon.com/images/P/0440416639.0...,156150,0,17
29158,0385335482,Confessions of a Shopaholic (Summer Display Op...,SOPHIE KINSELLA,2001,Delta,http://images.amazon.com/images/P/0385335482.0...,78783,8,120


In [57]:
final_rating = final_rating[final_rating['Number_of_Ratings'] >= 50]
print(final_rating.shape)
final_rating.sample(5)

(62167, 9)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,User-ID,Book-Rating,Number_of_Ratings
3700,0316969680,The Beach House,James Patterson,2002,"Little, Brown",http://images.amazon.com/images/P/0316969680.0...,178199,0,142
127394,0312974256,Welcome to Temptation,Jennifer Crusie,2001,St. Martin's Paperbacks,http://images.amazon.com/images/P/0312974256.0...,98741,0,77
2547,0140067477,The Tao of Pooh,Benjamin Hoff,1983,Penguin Books,http://images.amazon.com/images/P/0140067477.0...,85426,0,77
68239,0553801376,One Door Away from Heaven,Dean R Koontz,2001,Bantam Books,http://images.amazon.com/images/P/0553801376.0...,187256,9,90
11303,0671867113,Moonlight Becomes You,Mary Higgins Clark,1997,Pocket,http://images.amazon.com/images/P/0671867113.0...,147847,8,60


In [60]:
#Only rows with the same values in both the 'User-ID' and 'Book-Title' columns will be considered duplicates.
final_rating.drop_duplicates(['User-ID', 'Book-Title'], inplace=True)
final_rating.shape

(60143, 9)

## PIVOT TABLE

In [63]:
pivot_table = final_rating.pivot_table(columns='User-ID', index='Book-Title', values='Book-Rating')
pivot_table

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
pivot_table.fillna(0, inplace=True)
pivot_table

User-ID,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
from scipy.sparse import csr_matrix
sparse_matrix = csr_matrix(pivot_table.values)

## MODEL BUILDING AND TRAINING

In [67]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')
model.fit(sparse_matrix)

NearestNeighbors(algorithm='brute')

In [73]:
distance, suggestion = model.kneighbors(pivot_table.iloc[37,:].values.reshape(1, -1), n_neighbors = 6)

In [80]:
distance,  suggestion

(array([[ 0.        , 28.01785145, 28.24889378, 28.61817604, 30.31501278,
         30.49590136]]),
 array([[ 37, 138, 206, 703, 746, 185]]))

In [81]:
for i in range(len(suggestion)):
  print(pivot_table.index[suggestion[i]])

Index(['Accident', 'Daddy', 'Fine Things', 'Vanished', 'Zoya', 'Exclusive'], dtype='object', name='Book-Title')


In [82]:
book_names = pivot_table.index

## SAVING THE MODEL

In [87]:
import pickle
pickle.dump(model, open('/content/saved_elements/model.pkl', 'wb'))
pickle.dump(book_names, open('/content/saved_elements/book_names.pkl', 'wb'))
pickle.dump(final_rating, open('/content/saved_elements/final_rating.pkl', 'wb'))
pickle.dump(pivot_table, open('/content/saved_elements/pivot_table.pkl', 'wb'))

In [88]:
from google.colab import files
files.download('/content/saved_elements/model.pkl')
files.download('/content/saved_elements/book_names.pkl')
files.download('/content/saved_elements/final_rating.pkl')
files.download('/content/saved_elements/pivot_table.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## RECOMMENDER

In [89]:
def recommender(book_name):
  book_id = np.where(pivot_table.index==book_name)[0][0]
  distance, suggestion = model.kneighbors(pivot_table.iloc[book_id,:].values.reshape(1, -1), n_neighbors = 6)

  for i in range(len(suggestion)):
    books = pivot_table.index[suggestion[i]]
    for j in books:
      print(j)

In [91]:
recommender('Harry Potter and the Chamber of Secrets (Book 2)')

Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Sorcerers Stone (Book 1)
Exclusive
Jacob Have I Loved
